## Clustering

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Clustering").getOrCreate()

In [ ]:
data = spark.read.csv(path = "/kaggle/input/pyspark-ml-clustering/seeds_dataset.csv",
                      header = True, inferSchema = True)

In [ ]:
data.show()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = data.columns,
                            outputCol = "features")

data = assembler.transform(data)

In [ ]:
data.show()

## StandardScaler
_**Documentación:** https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StandardScaler.html_

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = "features",
                        outputCol = "scaled_features",
                        withStd = True,
                        withMean = False)

data = scaler.fit(data).transform(data)

data.show()

In [ ]:
data.select("scaled_features").show(truncate = False)

## K-Means

_**Documentación:** https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.KMeans.html_

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol = "scaled_features",
                predictionCol = "cluster", 
                k = 3,
                distanceMeasure = "euclidean")

model = kmeans.fit(data)

In [ ]:
cluster = model.transform(data)

cluster.show()

In [ ]:
centers = model.clusterCenters()
centers

In [ ]:
summary = model.summary

# Inercia: Suma de las distancias al cuadrado de todos los puntos con su centroide mas cercano
summary.trainingCost

### Ejercicio:
- Hacer el codigo para el método del codo para **KMeans** en PySpark.

In [ ]:
inercias = list()

for k in range(2, 11):
    kmeans = KMeans(featuresCol = "scaled_features",
                    predictionCol = "cluster", 
                    k = k,
                    distanceMeasure = "euclidean")

    model = kmeans.fit(data)
    
    inercia = model.summary.trainingCost
    
    inercias.append(inercia)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(2, 11), inercias)
plt.xlabel("K's")
plt.ylabel("Inercias")
plt.title("Elbow's Method")
plt.show()

## Bisecting K-Means

**Bisecting K-means** es una variante de **K-Means** que busca dividir el conjunto de datos en clusters de manera jerárquica, en lugar de buscar una partición óptima de los datos. 

**Bisecting K-Means** se puede resumir en los siguientes pasos:

1. Inicializar el algoritmo con un cluster que contenga todos los datos.

2. Mientras no se hayan alcanzado el número de clusters deseados:

    - Seleccionar el cluster más grande y dividirlo en dos sub-clusters utilizando el algoritmo **K-Means** con **k = 2**.

    - Seleccionar el sub-cluster que tenga la mayor suma de cuadrados de las distancias de cada punto al centroide del sub-cluster (Inercia) y repetir el paso 2 hasta alcanzar el número deseado de clusters.

**Bisecting K-Means** tiene la ventaja de ser más resistente a la inicialización aleatoria de los centroides, ya que cada división del cluster seleccionado se realiza utilizando el algoritmo **K-Means**, que tiene su propia inicialización aleatoria. Además, el algoritmo de **Bisecting K-Means** puede ser más eficiente que el algoritmo **K-Means** en conjuntos de datos con muchos puntos y/o muchas dimensiones, ya que divide el conjunto de datos en subconjuntos más pequeños antes de aplicar el algoritmo **K-Means**.

_**Documentación**: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.clustering.BisectingKMeans.html_

In [ ]:
from pyspark.ml.clustering import BisectingKMeans

bkmeans = BisectingKMeans(featuresCol    = "scaled_features",
                         predictionCol   = "cluster", 
                         k               = 3,
                         distanceMeasure = "euclidean")

model = bkmeans.fit(data)

In [ ]:
cluster = model.transform(data)

cluster.show()

In [ ]:
centers = model.clusterCenters()
centers

In [ ]:
summary = model.summary

# Inercia: Suma de las distancias al cuadrado de todos los puntos con su centroide mas cercano
summary.trainingCost

### Ejercicio:
- Hacer el codigo para el método del codo para **Bisecting K-Means** en PySpark.

In [ ]:
binercias = list()

for k in range(2, 11):
    kmeans = BisectingKMeans(featuresCol     = "scaled_features",
                             predictionCol   = "cluster", 
                             k               = k,
                             distanceMeasure = "euclidean")

    model = kmeans.fit(data)
    
    binercia = model.summary.trainingCost
    
    binercias.append(binercia)

In [ ]:
plt.plot(range(2, 11), binercias)
plt.xlabel("K's")
plt.ylabel("Inercias")
plt.title("Elbow's Method")
plt.show()

### Comparación

In [ ]:
plt.plot(range(2, 11), inercias, color = "red", label = "K-Means")
plt.plot(range(2, 11), binercias, color = "blue", label = "Bisecting K-Means")
plt.xlabel("K's")
plt.ylabel("Inercias")
plt.title("Elbow's Method")
plt.legend()
plt.show()

In [ ]:
################################################################################################################################